# Notebook to run gPhoton
Run gPhoton to create GALEX light curves etc.
Setup your source parameter and setting in the cell below.

In [1]:
region_name = "ALL_10-800" 
outdir = "./resources/gPhoton_out/"
region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+"_cat.fits"

srcs_ids = [55034 ,61491 ,64360 ,73354 ,76243 ,92867 ,104078 ,106745 ,226975 ,267514 ,588567 ,599887 ,] # WDs ALL

In [2]:
%matplotlib widget
import ipywidgets as widgets
import gPhoton
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import ascii
from astropy.table import Table
from astropy import units as uu
import os
from vasca.utils import mag2flux, flux2mag, tgalex_to_astrotime
import vasca.visualization as vvis
from vasca.region import Region
from vasca.resource_manager import ResourceManager

rg = Region()
rg.load_from_fits(region_fname)

#Subselect sources based on choice
if len(srcs_ids)>0:
    rg.tt_sources.add_index("rg_src_id")
    idx_srcs = rg.tt_sources.loc_indices["rg_src_id", srcs_ids]
    tt_srcs = Table(rg.tt_sources[idx_srcs])
else:
    tt_srcs = rg.tt_sources

rm = ResourceManager()
gphot_dir = rm.get_path("gal_gphoton", "sas_cloud")

display(tt_srcs)

fd_src_id,nr_det,ra,dec,pos_err,pos_xv,pos_var,pos_cpval,pos_rchiq,coadd_src_id,coadd_dist,obs_filter_id,sel,flux,flux_err,flux_nxv,flux_var,flux_cpval,flux_rchiq,coadd_ffactor,coadd_fdiff_s2n,rg_fd_id,rg_src_id,nr_fd_srcs,hr,hr_err,otype,main_id,otypes,z_value,distance_distance,distance_unit,distance_result
,,deg,deg,arcsec,arcsec2,arcsec2,,,,arcsec,,,1e-06 Jy,1e-06 Jy,,1e-12 Jy2,,,,,,,,,,,,,,,,arcsec
int32,int32[2],float64,float64,float32,float32,float32,float32,float32,int64,float32,int32[2],bool,float32[2],float32[2],float32[2],float32[2],float32[2],float32[2],float32[2],float32[2],int64,int32,int32,float32,float32,bytes32,bytes32,bytes32,float64,float64,bytes4,float32
-1,14 .. 4,348.90357602965054,19.41375292102418,0.12961677,-0.16667423,0.1757124,0.9601939,0.38698718,178527,0.27343646,1 .. 2,True,127.17284 .. 38.594788,0.6384927 .. 1.9439608,0.0018059316 .. 0.0030419875,36.337494 .. 59.990253,1.3913134e-13 .. 0.45650917,6.9238577 .. 0.86853236,1.002267 .. 1.0705498,0.35846016 .. 1.2124124,-1,55034,2,0.31256443,0.016097367,WD?,SDSS J231536.88+192448.9,blu|Pu*|*|Opt|WD?|WD?,--,168.9270,pc,0.4557
-1,7 .. 2,334.6190008630954,-0.00334150877900888,0.18584667,-0.18340522,0.12893102,0.8110655,0.40349314,198217,0.26084808,1 .. 2,True,121.26046 .. 42.269413,0.7279036 .. 0.8880146,0.0016064999 .. 0.005644754,27.450546 .. 11.66761,2.65163e-07 .. 0.00661471,6.8656754 .. 7.3747196,0.9975268 .. 1.0487088,-0.38367364 .. 1.7196137,-1,61491,2,0.35823846,0.011588516,WD*,PB 5130,WD*|WD*|WD*|WD*|*|Opt|UV|WD?|WD?,1.3299500,121.7450,pc,0.3802
-1,32 .. 4,333.54135905569626,0.8794295342456827,0.106176615,-0.26969245,0.14336883,0.9997828,0.34498087,208406,0.11125645,1 .. 2,True,19.849901 .. 5.189896,0.17154492 .. 0.40037686,0.06351051 .. -0.020882767,26.03123 .. 0.08260181,0.0 .. 0.9400651,26.48837 .. 0.13361953,0.94128394 .. 1.0394988,-5.53346 .. 0.3762283,-1,64360,2,0.34424755,0.02826839,WD*,GALEX J221409.8+005245,WD*|WD*|WD*|*|Opt|UV,--,439.0000,pc,0.2767
-1,18 .. 6,303.01100514369136,-48.394108786088324,0.11078485,-0.1118269,0.21495172,0.59480566,0.64826185,236659,0.16462971,1 .. 2,True,145.71655 .. 53.254395,0.65051293 .. 1.6317714,0.011681384 .. 0.09355949,262.29507 .. 336.1793,0.0 .. 4.538933e-19,41.07661 .. 19.113188,0.9875043 .. 0.92640793,-2.2628753 .. -1.732858,-1,73354,2,0.38933843,0.012308751,WD?,Gaia DR3 6669015352243852800,*|WD?,--,--,--,0.3998
-1,25 .. 12,261.0003596219375,60.74772407491583,0.09249079,-0.12485813,0.21191294,0.78836334,0.5457976,245032,0.072778955,1 .. 2,True,43.02675 .. 51.450314,0.27040577 .. 0.68775964,0.021784516 .. 0.014586984,42.229954 .. 74.628174,0.0 .. 9.499622e-33,27.235014 .. 16.378187,0.9762591 .. 0.9339476,-2.9283037 .. -4.3375144,-1,76243,2,1.0981839,0.017458435,WD?,ZTF J172400.04+604451.8,EB*|*|WD?|WD?,--,--,--,0.2700
-1,15 .. -1,244.14364475974057,54.16992508668509,0.14302015,-0.149046,0.15964259,0.6340721,0.52604604,71751,0.20260328,1 .. 0,False,43.955715 .. -1.0,0.373552 .. -1.0,0.023890479 .. -100.0,48.38352 .. -1.0,0.0 .. -1.0,23.603268 .. -1.0,0.990053 .. -100.0,-1.0570939 .. -10000.0,-1,92867,2,-1.0,-1.0,WD?,GALEX J161634.4+541011,*|*|Opt|UV|WD?|WD?,--,93.6920,pc,1.0158
-1,21 .. 1,219.22854193199083,34.06670048559544,0.11800814,-0.10925614,0.2025599,0.68355554,0.61700284,351692,0.035019293,1 .. 2,True,30.282242 .. 37.433167,0.3364404 .. 3.059399,0.013331741 .. -100.0,19.445185 .. -100.0,4.734923e-28 .. -1.0,9.079622 .. -1.0,1.0011699 .. 0.9832684,0.090507984 .. -0.18113555,-1,104078,2,1.1301088,0.1134821,WD?,GALEX J143654.8+340400,*|UV|WD?|WD?,--,--,--,0.2658
-1,18 .. 8,218.05243811169709,35.840903328171464,0.12735511,-0.29414088,0.17379755,0.96632844,0.39901668,361741,0.07959464,1 .. 2,True,14.915235 .. 6.4086227,0.24988876 .. 0.38860035,0.04386847 .. 0.04654254,12.461055 .. 3.405273,3.1348733e-25 .. 0.030883376,9.390375 .. 2.204061,0.95756274 .. 0.97631735,-2.0785728 .. -0.309062,-1,106745,2,0.32614058,0.026615512,WD?,Gaia DR2 1480420251349953920,*|WD?,--,--,--,0.2143


### Do aperture photometry for each visit with gPhoton

In [3]:
def get_lc(filename, sig_cut=3.5):
    #Load file
    dd_gph = np.load(filename,allow_pickle='TRUE').item()

    # Get gphoton lc
    keep_keys = ('t_mean', 'exptime','flux_bgsub', 'flux_bgsub_err','mag_bgsub','mag_bgsub_err_2',"flags","mag_mcatbgsub","mag_mcatbgsub_err_2", "mag", "flags")
    dd_gap = {x: dd_gph["gAperture"][x] for x in keep_keys if x in dd_gph["gAperture"]}
    dd_gap["s2n"]=dd_gap['flux_bgsub']/dd_gap['flux_bgsub_err']
    
    # Units of flux_bgsub are in erg sec^-1 cm^-2 Å^-1. . Get also Jy flux from AB magnitude
    dd_gap['flux'],dd_gap['flux_err'] = mag2flux(dd_gap["mag_mcatbgsub"],dd_gap["mag_mcatbgsub_err_2"])
    dd_gap['t_mean']= tgalex_to_astrotime(dd_gap['t_mean'],"mjd")
    # Units of time are in "GALEX Time" = "UNIX Time" - 315964800, change to MJD
    
    tt_gap = Table(data= dd_gap)
    sel_gap = (tt_gap["s2n"]>sig_cut) * (tt_gap["flags"]<0.5)
    return tt_gap,sel_gap

def plot_lc(rg_src_id):

    #Get table entries for that source
    tt_src = tt_srcs[tt_srcs["rg_src_id"]==rg_src_id]

    fname = gphot_dir+"/gPhoton_ra"+str(round(tt_src["ra"][0],5))+"_dec"+str(round(tt_src["dec"][0],5))+"_nuv_app.npy"
    tt_gap, sel_gap = get_lc(fname)

    fig_lc_src = plt.figure("Light curve",figsize=(12, 8), clear=True)
    vvis.plot_light_curve(rg, rg_src_ids = rg_src_id,fig=fig_lc_src, flux_var="flux", markersize=10) #

    #plt.errorbar(tt_gap['t_mean'],1.132*tt_gap['flux'],yerr=tt_gap['flux_err'],fmt="k*", label="gPhoton al l", alpha=0.3)
    plt.plot(tt_gap['t_mean'][sel_gap],1.120*tt_gap['flux'][sel_gap], lw=0,marker="o", color="k",label="gPhoton sel") #,yerr=tt_gap['flux_err'][sel_gap],
    plt.yscale('log')

    #Add FUV if existent
    fname_fuv = fname.replace("_nuv","_fuv")
    if os.path.isfile(fname_fuv):
        tt_gap_fuv,sel_gap_fuv = get_lc(fname_fuv)
        #plt.errorbar(tt_gap_fuv['t_mean'],1.132*tt_gap_fuv['flux'],yerr=tt_gap_fuv['flux_err'],fmt="ko", label="gPhoton all", alpha=0.3)
        plt.plot(tt_gap_fuv['t_mean'][sel_gap_fuv],1.120*tt_gap_fuv['flux'][sel_gap_fuv],label="gPhoton sel", lw= 0, marker="o", markerfacecolor=None, markeredgecolor="k") #,yerr=tt_gap_fuv['flux_err'][sel_gap_fuv]

    plt.legend()
    plt.show()
    display(tt_src)
    display(tt_gap)
                  
widgets.interactive( plot_lc, rg_src_id=tt_srcs["rg_src_id"])

interactive(children=(Dropdown(description='rg_src_id', options=(55034, 61491, 64360, 73354, 76243, 92867, 104…

In [4]:
# Calculate apperture flux correctin from http://www.galex.caltech.edu/researcher/techdoc-ch5.html
f1 = mag2flux(19)
f2 = mag2flux(19-0.20)
mag = flux2mag(f2)
print(f1,f2,f2/f1, mag)


91.20108393559062 1e-06 Jy 109.64781961431808 1e-06 Jy 1.202264434617413 18.8 mag(AB)


In [5]:
1/(60*60)

0.0002777777777777778